In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

print("\n====================================")
print(" LOADING DATA")
print("====================================")

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(f"Train Shape: {train.shape}")
print(f"Test Shape : {test.shape}")

print("\nPreview:")
print(train.head())


 LOADING DATA
Train Shape: (181507, 279)
Test Shape : (77789, 278)

Preview:
       id      full_sq      life_sq      floor product_type   sub_area  \
0  106299    -0.280658    50.127224  10.970619       Type_A   Area_380   
1  125559  2129.178675  3438.561939   6.282464       Type_B  Area_1375   
2  204969    44.255548   -15.662341   0.894701       Type_A   Area_272   
3  248026  2622.821354  1373.058212  15.971260       Type_B  Area_1750   
4   51881    52.877556    12.392606   2.876796       Type_A  Area_1773   

   green_zone_part  indust_part  children_preschool  \
0         0.099506     0.222095         6255.829172   
1         0.588042     0.157862         7391.899017   
2         0.074837     0.266754         5658.091711   
3         0.850289     0.393528        13262.734984   
4         0.070450     0.238344         3842.416286   

   preschool_education_centers_raion  ...  office_sqm_5000_log  \
0                           4.076307  ...            13.558453   
1             

In [4]:
# =========================================================
# 1. Missing Values Analysis
# =========================================================
print("\n====================================")
print("📌 MISSING VALUE ANALYSIS")
print("====================================")

missing_vals = train.isna().sum()
missing_percent = (missing_vals / len(train)) * 100

missing_df = pd.DataFrame({
    "Missing Count": missing_vals,
    "Missing %": missing_percent
}).sort_values(by="Missing %", ascending=False)

print("\nMissing Values Summary:")
print(missing_df)

print("\nFeatures with > 40% missing values:")
print(missing_df[missing_df["Missing %"] > 40]) 


📌 MISSING VALUE ANALYSIS

Missing Values Summary:
                              Missing Count  Missing %
office_sqm_5000_log                   11249   6.197557
trc_sqm_5000_log                       7216   3.975604
full_all_log                           4023   2.216443
sport_count_5000_log                   2351   1.295267
area_m_log                               28   0.015426
...                                     ...        ...
culture_objects_top_25_raion              0   0.000000
shopping_centers_raion                    0   0.000000
office_raion                              0   0.000000
thermal_power_plant_raion                 0   0.000000
mosque_count_5000                         0   0.000000

[279 rows x 2 columns]

Features with > 40% missing values:
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []


In [5]:
# =========================================================
# 2. Target Variable Analysis (Check Imbalance)
# =========================================================

TARGET = "price_doc"    # ← change according to your dataset!

if TARGET in train.columns:
    print("\n====================================")
    print("📌 TARGET VARIABLE ANALYSIS")
    print("====================================")

    print(f"Target Stats:\n{train[TARGET].describe()}")
    
    # Check if skewed
    print("\nSkewness of Target:", train[TARGET].skew())

   

else:
    print("\n⚠ ALERT: No 'target' column found. Set TARGET variable correctly.")


📌 TARGET VARIABLE ANALYSIS
Target Stats:
count    181507.000000
mean         14.845599
std          21.533138
min           0.392328
25%           5.303449
50%           7.186257
75%          11.781645
max         109.864990
Name: price_doc, dtype: float64

Skewness of Target: 2.7868141322285016


In [6]:
# =========================================================
# 3. Basic Statistical Summary
# =========================================================
print("\n====================================")
print("📌 BASIC STATISTICAL SUMMARY")
print("====================================")

print(train.describe().T)



📌 BASIC STATISTICAL SUMMARY
                      count           mean           std           min  \
id                 181507.0  129814.082570  74904.378967      2.000000   
full_sq            181507.0     446.686938   1106.420198    -60.517111   
life_sq            181507.0     595.833855   1511.699330    -98.221082   
floor              181507.0      11.597576     14.755608     -0.645939   
green_zone_part    181507.0       0.240568      0.213208     -0.013725   
...                     ...            ...           ...           ...   
high_floor         181507.0       0.166980      0.372959      0.000000   
large_apartment    181507.0       0.174302      0.379370      0.000000   
rooms_inferred     181507.0      39.677136    100.601553      1.000000   
living_efficiency  181507.0       2.081635     87.664118 -15137.795627   
price_doc          181507.0      14.845599     21.533138      0.392328   

                            25%            50%            75%            max  
id 

In [7]:
# 4. Correlation Matrix
# =========================================================
print("\n====================================")
print("📌 CORRELATION ANALYSIS")
print("====================================")

# Only numeric features
num_cols = train.select_dtypes(include=[np.number]).columns

corr_matrix = train[num_cols].corr()

# Show top correlated features with target
if TARGET in num_cols:
    print("\nTop 20 correlations with Target:")
    print(corr_matrix[TARGET].sort_values(ascending=False).head(20))





📌 CORRELATION ANALYSIS

Top 20 correlations with Target:
price_doc                     1.000000
large_apartment               0.737889
full_sq                       0.669290
leisure_count_1000            0.659987
leisure_count_500             0.659467
cafe_count_500_price_4000     0.654416
trc_sqm_500                   0.651915
cafe_count_1000_price_high    0.651016
mosque_count_500              0.649162
cafe_count_500_price_high     0.648583
cafe_count_500_price_1000     0.646469
leisure_count_1500            0.640853
cafe_count_1000_price_2500    0.638695
cafe_count_1000_price_4000    0.638275
office_sqm_1000               0.637163
cafe_count_500_price_1500     0.636903
cafe_count_1500_price_high    0.636007
office_count_500              0.635852
big_church_count_1000         0.635598
big_church_count_500          0.635320
Name: price_doc, dtype: float64


In [8]:
# Identify target column (adjust based on your data)
target_col = train.columns[-1]  # Assuming last column is target
print(f"✓ Target column: '{target_col}'")

✓ Target column: 'price_doc'


In [9]:
# Separate features and target
X = train.drop(columns=[target_col])
y = train[target_col]


# Drop ID column if present (usually first column)
if 'id' in X.columns.str.lower():
    id_col = [col for col in X.columns if 'id' in col.lower()][0]
    X = X.drop(columns=[id_col])
    test_df = test.drop(columns=[id_col])

print(f"\n✓ Features shape: {X.shape}")
print(f"✓ Target shape: {y.shape}")


✓ Features shape: (181507, 277)
✓ Target shape: (181507,)


In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

print("\n" + "="*80)
print("STEP 2: PREPROCESSING PIPELINE")
print("="*80)

def preprocess_data(train, test, target_col):
    """
    Comprehensive preprocessing pipeline
    """
    
    # Separate target
    X_train = train.drop(columns=[target_col])
    y_train = train[target_col]
    X_test = test.copy()
    
    # Drop ID column
    id_cols = [col for col in X_train.columns if 'id' in col.lower()]
    for col in id_cols:
        X_train = X_train.drop(columns=[col])
        if col in X_test.columns:
            X_test = X_test.drop(columns=[col])
    
    print("\n--- Feature Types ---")
    numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    
    print(f"Numerical: {len(numerical_features)}")
    print(f"Categorical: {len(categorical_features)}")


    # ========================================================================
    # HANDLE MISSING VALUES
    # ========================================================================
    print("\n--- Handling Missing Values ---")
    
    # Numerical features: impute with median
    for col in numerical_features:
        if X_train[col].isnull().sum() > 0:
            median_val = X_train[col].median()
            X_train[col].fillna(median_val, inplace=True)
            X_test[col].fillna(median_val, inplace=True)
            print(f"  ✓ Imputed '{col}' with median: {median_val:.2f}")
    
    # Categorical features: impute with mode
    for col in categorical_features:
        if X_train[col].isnull().sum() > 0:
            mode_val = X_train[col].mode()[0]
            X_train[col].fillna(mode_val, inplace=True)
            X_test[col].fillna(mode_val, inplace=True)
            print(f"  ✓ Imputed '{col}' with mode: {mode_val}")




    # ========================================================================
    # HANDLE CATEGORICAL FEATURES
    # ========================================================================
    print("\n--- Encoding Categorical Features ---")
    
    if len(categorical_features) > 0:
        # Check cardinality
        high_cardinality = []
        low_cardinality = []
        for col in categorical_features:
            unique_count = X_train[col].nunique()
            if unique_count > 10:
                high_cardinality.append(col)
            else:
                low_cardinality.append(col)
        
        # One-hot encode low cardinality features
        if len(low_cardinality) > 0:
            print(f"\n  One-hot encoding {len(low_cardinality)} features...")
            X_train = pd.get_dummies(X_train, columns=low_cardinality, drop_first=True)
            X_test = pd.get_dummies(X_test, columns=low_cardinality, drop_first=True)
        
        # Label encode high cardinality features
        if len(high_cardinality) > 0:
            print(f"  Label encoding {len(high_cardinality)} features...")
            for col in high_cardinality:
                le = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
                X_train[col] = le.fit_transform(X_train[[col]].astype(str)).ravel()
                X_test[col] = le.transform(X_test[[col]].astype(str)).ravel()
    
    # Align columns between train and test
    missing_cols = set(X_train.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0
    
    X_test = X_test[X_train.columns]
    
    print(f"\n✓ Final feature shape: {X_train.shape}")



    # ========================================================================
    # FEATURE ENGINEERING
    # ========================================================================
    print("\n--- Feature Engineering ---")
    
    # Get numerical columns after encoding
    numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Create polynomial features for top correlated features with target
    if len(numerical_cols) > 0 and len(numerical_cols) <= 20:
        # Calculate correlations
        temp_df = X_train.copy()
        temp_df['target'] = y_train
        correlations = temp_df.corr()['target'].drop('target').abs().sort_values(ascending=False)
        
        # Take top 3 features
        top_features = correlations.head(3).index.tolist()
        
        for i, feat in enumerate(top_features[:2]):
            # Squared term
            X_train[f'{feat}_squared'] = X_train[feat] ** 2
            X_test[f'{feat}_squared'] = X_test[feat] ** 2
            print(f"  ✓ Created '{feat}_squared'")
        
        # Interaction between top 2
        if len(top_features) >= 2:
            feat1, feat2 = top_features[0], top_features[1]
            X_train[f'{feat1}_{feat2}_interaction'] = X_train[feat1] * X_train[feat2]
            X_test[f'{feat1}_{feat2}_interaction'] = X_test[feat1] * X_test[feat2]
            print(f"  ✓ Created interaction: {feat1} × {feat2}")



            # ========================================================================
    # SCALING
    # ========================================================================
    print("\n--- Scaling Features ---")
    
    scaler = RobustScaler()  # Robust to outliers
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    print("✓ Features scaled using RobustScaler")
    
    return X_train_scaled, X_test_scaled, y_train

# Preprocess data
X_processed, X_test_processed, y_processed = preprocess_data(train, test, target_col)


STEP 2: PREPROCESSING PIPELINE

--- Feature Types ---
Numerical: 255
Categorical: 15

--- Handling Missing Values ---


C:\Users\sagar\AppData\Local\Temp\ipykernel_16352\2365127932.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(median_val, inplace=True)
C:\Users\sagar\AppData\Local\Temp\ipykernel_16352\2365127932.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

  ✓ Imputed 'area_m_log' with median: 16.04
  ✓ Imputed 'raion_popul_log' with median: 11.33
  ✓ Imputed 'full_all_log' with median: 11.48
  ✓ Imputed 'office_sqm_5000_log' with median: 13.95
  ✓ Imputed 'trc_sqm_5000_log' with median: 14.15
  ✓ Imputed 'sport_count_5000_log' with median: 4.16

--- Encoding Categorical Features ---

  One-hot encoding 14 features...
  Label encoding 1 features...

✓ Final feature shape: (181507, 273)

--- Feature Engineering ---

--- Scaling Features ---
✓ Features scaled using RobustScaler


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import cross_val_score


print("\n" + "="*80)
print("STEP 3: GRADIENT BOOSTING REGRESSOR")
print("="*80)

# ==========================================
# BASE MODEL (check baseline performance)
# ==========================================
gbr = GradientBoostingRegressor(random_state=42)

scores = cross_val_score(gbr, X_processed, y_processed, cv=5, scoring='neg_root_mean_squared_error')
print(f"Baseline CV RMSE: {-scores.mean():.4f}")


STEP 3: GRADIENT BOOSTING REGRESSOR


In [ ]:
# ===============================
# 2. Generate Kaggle Submission
# ===============================
# Predict on test set
y_test_pred =gbr.predict(X_test_processed)

# Prepare submission DataFrame
submission = pd.DataFrame({
    'id': test['id'],   # Assuming test.csv has 'id' column
    'price_doc': y_test_pred
})

# Save to CSV
submission.to_csv("gradient_boosting_regressor_submission.csv", index=False)
print("Kaggle submission saved: gradient_boosting_regressor_submission.csv")

Kaggle submission saved: regression_tree_random_forest_submission.csv


In [ ]:
# ==========================================
# HYPERPARAMETER TUNING (GridSearch)
# ==========================================
param_grid_gbr = {
    "n_estimators": [200, 500, 800],
    "learning_rate": [0.01, 0.03, 0.05],
    "max_depth": [3, 4, 5],
    "subsample": [0.8, 1.0],
    "min_samples_split": [2, 5, 10]
}
# 3*3*3*2*3 = 324 combinations
#  5*324=1620 fits with cv=5

grid_gbr = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid_gbr,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_gbr.fit(X_processed, y_processed)

print("\nBest Gradient Boosting RMSE:", -grid_gbr.best_score_)
print("Best Params:", grid_gbr.best_params_)


In [ ]:
# ==========================================
# TRAIN BEST MODEL ON FULL TRAINING SET
# ==========================================
best_gbr = grid_gbr.best_estimator_
best_gbr.fit(X_processed, y_processed)


# ==========================================
# GENERATE TEST PREDICTIONS (for Kaggle)
# ==========================================
gbr_preds = best_gbr.predict(X_test_processed)

submission = pd.DataFrame({
    "id": test["id"],   # Change to correct ID column name
    "price_doc": gbr_preds
})

submission.to_csv("gbr_submission.csv", index=False)
print("\n✓ Gradient Boosting submission file created: gbr_submission.csv")